3D laplace equation using gmsh

In [1]:
import gmsh
import numpy as np
from dolfinx.io import (VTXWriter, distribute_entity_data, gmshio)
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot, nls, log, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import (Constant, Function, functionspace,
                         assemble_scalar, dirichletbc, form, locate_dofs_topological, set_bc)
from ufl import ds, dx, grad, inner
from mpi4py import MPI
from petsc4py import PETSc
from basix.ufl import element
from dolfinx.mesh import create_mesh, meshtags_from_entities

Initialize Domian Boundary and Lengths

In [2]:
gmsh.initialize()

x_L = y_L = z_L = 30 # Length of the cube sides
c_x = x_L/2 # x center
c_y = y_L/2 # y center
c_z = z_L/2 # z center
r = 1 # Sphere radius
gdim = 3
mesh_comm = MPI.COMM_WORLD
model_rank = 0
if mesh_comm.rank == model_rank:
    box = gmsh.model.occ.addBox(0, 0, 0, x_L, y_L, z_L, tag=0)
    sphere = gmsh.model.occ.addSphere(c_x, c_y, c_z, r, tag=1)

Meshing can only be done on a single core, use the zeroth (first) core to make the mesh

In [3]:
if mesh_comm.rank == model_rank:
    fluid = gmsh.model.occ.cut([(gdim, box)], [(gdim, sphere)])
    gmsh.model.occ.synchronize()

Add the fuild_marker which allows meshing of interior nodes 

In [4]:
fluid_marker = 1
if mesh_comm.rank == model_rank:
    volumes = gmsh.model.getEntities(dim=gdim)
    assert (len(volumes) == 1)
    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker)
    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid")

<!-- surfaces = gmsh.model.occ.getEntities(dim=2)
inlet_marker, outlet_marker, top_wall_marker, bottom_wall_marker, left_wall_marker, right_wall_marker, obstacle_marker = 2, 3, 4, 5, 6, 7, 8
walls = []
sphere = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [x_L/2, y_L/2, 0]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], inlet_marker)
        inlet = surface[1]
        gmsh.model.setPhysicalName(surface[0], inlet_marker, "Fluid inlet")
    elif np.allclose(com, [x_L/2, y_L/2, z_L]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], outlet_marker)
        gmsh.model.setPhysicalName(surface[0], outlet_marker, "Fluid outlet")
    elif (
        np.isclose(com[2], 0)
        or np.isclose(com[1], 30)
        or np.isclose(com[2], 30)
        or np.isclose(com[1], 0)
    ):
        walls.append(surface[1])
    else:
        sphere.append(surface[1])
gmsh.model.addPhysicalGroup(2, walls, wall_marker)
gmsh.model.setPhysicalName(2, wall_marker, "Walls")
gmsh.model.addPhysicalGroup(2, obstacles, obstacle_marker)
gmsh.model.setPhysicalName(2, obstacle_marker, "Obstacle") -->

In [5]:

inlet_marker, outlet_marker, top_wall_marker, bottom_wall_marker, right_wall_marker, left_wall_marker, sphere_marker = 2, 3, 4, 5, 6, 7, 8
inlet, outlet, topwall, bottomwall, rightwall, leftwall, sphere = [], [], [], [], [], [], []
if mesh_comm.rank == model_rank:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        if np.allclose(center_of_mass, [x_L/2, y_L/2, 0]):
            inlet.append(boundary[1])
        elif np.allclose(center_of_mass, [x_L/2, y_L/2, z_L]):
            outlet.append(boundary[1])
        elif np.allclose(center_of_mass, [x_L/2, y_L, z_L/2]):
            topwall.append(boundary[1])
        elif np.allclose(center_of_mass, [x_L/2, 0, z_L/2]):
            bottomwall.append(boundary[1])
        elif np.allclose(center_of_mass, [0, y_L/2, z_L/2]):
            leftwall.append(boundary[1])
        elif np.allclose(center_of_mass, [x_L, y_L/2, z_L/2]):
            rightwall.append(boundary[1])
        else:
            sphere.append(boundary[1])
    gmsh.model.addPhysicalGroup(2, topwall, top_wall_marker)
    gmsh.model.setPhysicalName(2, top_wall_marker, "Top_Wall")
    gmsh.model.addPhysicalGroup(2, bottomwall, bottom_wall_marker)
    gmsh.model.setPhysicalName(2, bottom_wall_marker, "Bottom_Wall")
    
    gmsh.model.addPhysicalGroup(2, inlet, inlet_marker)
    gmsh.model.setPhysicalName(2, inlet_marker, "Inlet")
    gmsh.model.addPhysicalGroup(2, outlet, outlet_marker)
    gmsh.model.setPhysicalName(2, outlet_marker, "Outlet")
    
    gmsh.model.addPhysicalGroup(2, leftwall, left_wall_marker)
    gmsh.model.setPhysicalName(2, left_wall_marker, "Left_Wall")
    gmsh.model.addPhysicalGroup(2, rightwall, right_wall_marker)
    gmsh.model.setPhysicalName(2, right_wall_marker, "Right_Wall")
    
    gmsh.model.addPhysicalGroup(2, sphere, sphere_marker)
    gmsh.model.setPhysicalName(2, sphere_marker, "Sphere")

Set max mesh size

In [6]:
res_min = 3
if mesh_comm.rank == model_rank:
    distance_field = gmsh.model.mesh.field.add("Distance")
    gmsh.model.mesh.field.setNumbers(distance_field, "EdgesList", sphere)
    threshold_field = gmsh.model.mesh.field.add("Threshold")
    gmsh.model.mesh.field.setNumber(threshold_field, "IField", distance_field)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", res_min)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 2)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", 0)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 0)
    min_field = gmsh.model.mesh.field.add("Min")
    gmsh.model.mesh.field.setNumbers(min_field, "FieldsList", [threshold_field])
    gmsh.model.mesh.field.setAsBackgroundMesh(min_field)

In [7]:
if mesh_comm.rank == model_rank:
    gmsh.model.mesh.generate(gdim)
    gmsh.model.mesh.setOrder(1)
    gmsh.model.mesh.optimize("Netgen")

Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 14 (Circle)
Info    : [ 30%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 60%] Meshing curve 21 (Line)
Info    : [ 70%] Meshing curve 22 (Line)
Info    : [ 70%] Meshing curve 23 (Line)
Info    : [ 80%] Meshing curve 24 (Line)
Info    : [ 90%] Meshing curve 25 (Line)
Info    : [ 90%] Meshing curve 26 (Line)
Info    : [100%] Meshing curve 27 (Line)
Info    : Done meshing 1D (Wall 0.000949791s, CPU 0.001087s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 7 (Sphere, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 9 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 10 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 11 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 12 (Plane, Fron

Info    : Done meshing 3D (Wall 0.190828s, CPU 0.187776s)
Info    : Optimizing mesh...
Info    : Optimizing volume 0
Info    : Optimization starts (volume = 26997.2) with worst = 0.010609 / average = 0.779907:
Info    : 0.00 < quality < 0.10 :        57 elements
Info    : 0.10 < quality < 0.20 :       125 elements
Info    : 0.20 < quality < 0.30 :       238 elements
Info    : 0.30 < quality < 0.40 :       350 elements
Info    : 0.40 < quality < 0.50 :       522 elements
Info    : 0.50 < quality < 0.60 :       929 elements
Info    : 0.60 < quality < 0.70 :      2202 elements
Info    : 0.70 < quality < 0.80 :      4546 elements
Info    : 0.80 < quality < 0.90 :      7672 elements
Info    : 0.90 < quality < 1.00 :      3892 elements
Info    : 415 edge swaps, 15 node relocations (volume = 26997.2): worst = 0.229109 / average = 0.792874 (Wall 0.00692871s, CPU 0.00625s)
Info    : 420 edge swaps, 17 node relocations (volume = 26997.2): worst = 0.282004 / average = 0.793063 (Wall 0.00826947s, 

Info    : Total badness = 26347.5 
Info    : SplitImprove 
Info    : badmax = 13.9571 
Info    : 0 splits performed 
Info    : ImproveMesh 
Info    : Total badness = 26347.5 
Info    : Total badness = 26182.3 
Info    : SwapImprove  
Info    : 1187 swaps performed 
Info    : SwapImprove2  
Info    : 0 swaps performed 
Info    : ImproveMesh 
Info    : Total badness = 24144.8 
Info    : Total badness = 23552.6 
Info    : CombineImprove 
Info    : 44 elements combined 
Info    : ImproveMesh 
Info    : Total badness = 23169.7 
Info    : Total badness = 23093.7 
Info    : SplitImprove 
Info    : badmax = 6.87274 
Info    : 0 splits performed 
Info    : ImproveMesh 
Info    : Total badness = 23093.7 
Info    : Total badness = 23081.5 
Info    : SwapImprove  
Info    : 167 swaps performed 
Info    : SwapImprove2  
Info    : 0 swaps performed 
Info    : ImproveMesh 
Info    : Total badness = 22987.4 
Info    : Total badness = 22886.9 
Info    : CombineImprove 
Info    : 8 elements combined 
In

In [8]:
gmsh.write("LaplaceEquationFE_Practice_4.msh")

Info    : Writing 'LaplaceEquationFE_Practice_4.msh'...
Info    : Done writing 'LaplaceEquationFE_Practice_4.msh'


In [9]:
mesh, _, ft = gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=gdim)
ft.name = "Facet markers"

Apply boundary condtions

In [10]:
msh = element("Lagrange", mesh.topology.cell_name(), 2)
V = fem.functionspace(mesh, msh)
fdim = mesh.topology.dim - 1

# Define boundary conditions

# # Inlet
# class InletVelocity():
#     def __call__(self, x):
#         values = np.zeros((1, x.shape[1]), dtype=PETSc.ScalarType)
#         values[0] = x[1] - L/2
#         return values


# u_inlet = fem.Function(V)
# inlet_velocity = InletVelocity()
# u_inlet.interpolate(inlet_velocity)
# bc_inflow = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(inlet_marker)))

# Outlet
# bc_outlet = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(outlet_marker)))

# Obstacle
bc_obstacle = dirichletbc(PETSc.ScalarType(0), locate_dofs_topological(V, fdim, ft.find(sphere_marker)), V)

# Top Wall
bc_top_wall = dirichletbc(PETSc.ScalarType(x_L/2), locate_dofs_topological(V, fdim, ft.find(top_wall_marker)), V)

# Bottom Wall
bc_bottom_wall = dirichletbc(PETSc.ScalarType(-x_L/2), locate_dofs_topological(V, fdim, ft.find(bottom_wall_marker)), V)

Define the problem

In [11]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(mesh,  default_scalar_type(0))
bcs = [bc_obstacle, bc_top_wall, bc_bottom_wall]
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

Solve the problem

In [12]:
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

Export data for post processing

In [13]:
# xyz = V.tabulate_dof_coordinates()
# x = xyz[:,0]
# y = xyz[:,1]
# z = xyz[:,2]
# print(x)
# sol = np.array((uh.vector))
# print(sol)
# np.savetxt('FE_Practice_4_x_Mesh.csv', x, delimiter=',')
# np.savetxt('FE_Practice_4_y_Mesh.csv', y, delimiter=',')
# np.savetxt('FE_Practice_4_z_Mesh.csv', z, delimiter=',')
# np.savetxt('FE_Practice_4_Sol.csv', sol, delimiter=',')

In [17]:
from dolfinx.io import XDMFFile
from basix.ufl import element as VectorElement
with XDMFFile(MPI.COMM_WORLD, "LaplaceEquation3D_FE_Practice_4.xdmf", "w") as pfile_xdmf:
    uh.x.scatter_forward()
    P3 = VectorElement("Lagrange", mesh.basix_cell(), 1)
    u1 = Function(functionspace(mesh, P3))
    u1.interpolate(uh)
    pfile_xdmf.write_mesh(mesh)
    pfile_xdmf.write_function(u1)